# INTEGRATED INFORMATION THEORY (IIT) 3.0 
# Non-binary systems 

by [Juan Gomez](https://github.com/juanogo)
/ [Original Paper](https://journals.plos.org/ploscompbiol/article/file?id=10.1371/journal.pcbi.1003588&type=printable) / [Videos on YouTube](https://www.youtube.com/watch?v=i3G-Wt5az30)

## Introduction

[PyPhi](http://integratedinformationtheory.org/calculate.html) is a software tool that calculates IIT in small binary systems but does not allow for non-binary systems calculations. This implementation is intended as an add-on to the original PyPhi to compensate for such lack.

In terms of knowledge gain, it would be of great interest to Tononi's lab, to see how the number of concepts, and phi values (small phi and Big Phi) compare between systems with similar numbers of states but different numbers of states per element.

## Imports

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance
import matplotlib.pyplot as plt
from itertools import product 
import string

## A non-binary system of three elements (candidate system1): 
![title](non-binary.png)


In [354]:
def calculate_state_system1(a_prev,b_prev,c_prev):
    """
    Calculates a new state of the candidate system1 described in the graph above:
    This function defines the behavior of the elements and the interaction of the whole system1.
    Given the previous state of the elements in the system1 (a_prev,b_prev,c_prev),
    it calculates and returns the new state of the system1's elements(a_next,b_next,c_next).
    Note: this function has to be re-defined every time the system changes. 
    """ 
        
    a_next= b_prev + c_prev
    if a_next > 0:
        a_next=1
    elif a_next< 0:
        a_next=-1
        
    b_next= a_prev + c_prev
    if b_next > 0:
        b_next=1
    elif b_next< 0:
        b_next=-1
        
    c_next= a_prev + b_prev
    if c_next > 0:
        c_next=1
    elif c_next< 0:
        c_next=-1
        
    return a_next, b_next, c_next

def xor(x,y):
    return 0 if ((x==0 and y==0) or (x==1 and y==1)) else  1

def calculate_state_system2(a_prev,b_prev,c_prev):
    """
    This is a regular binary system (described in the paper) just for testing the code.
    """ 
    a_next= b_prev or c_prev
    b_next= a_prev and c_prev
    c_next= xor(a_prev,b_prev)
    return a_next, b_next, c_next



In [353]:
def perturbate(system,prev_states):
    """
    This function perturbates a system 'system', given the set of system's elements previous states 'prev_states'.
    Then, it returns the next states of the system 'system'.
    For each new candidate system this function needs to expand its options: system==1, system==2, etc.
    Also, for each option "system==n" added, a function "calculate_state_system n" must be created above.
    Example: perturbate(1,[-1,0,1]) perturbates system1 (system==1), in state [-1,0,1]
    """ 
   
    if system==1:
        return calculate_state_system1(prev_states[0],prev_states[1],prev_states[2])
    elif system==2:
        return calculate_state_system2(prev_states[0],prev_states[1],prev_states[2])
    else:
        print("System not recognized ")


## TPM (Transition Probability Matrix) calculation:

A matrix that specifies the probability with which any state of a system transitions to any other system state. The TPM is determined by the mechanisms of a system and obtained by perturbing the system into all its possible states.

In [235]:
def tpm(system,number_nodes,states):
    """
    This function returns the TPM of a system "system", given its number of nodes and the possible states
    that a node in the system can take.
    Example: tpm(1,3,[-1,0,1]) returns the TPM of the system=1 (described above), whose 3 nodes can have each, one of
    three states: -1, 0 or 1
    """ 
    
    number_states=len(states)**number_nodes
    matrix=np.zeros((number_states,number_states))
    
    all_pos_states=list(product(states, repeat=number_nodes))
    all_pos_states=list(map(list, list(zip(*all_pos_states))[::-1]))

    #goes through all possible states and perturbates the system with all of them, one by one 
    for i in range(number_states):
        #next_state is the next state obtained by perturbating the system with the i-th state 
        next_state=list(perturbate(system,[row[i] for row in all_pos_states]))
        #goes through all possible states to relate them (0 or 1) with the one obtained by the perturbation
        for j in range(number_states):
            state=[row[j] for row in all_pos_states]
            if next_state==state:
                matrix[i][j]=1 #100% probabilty of this being the state obtained by the perturbation
            else:
                matrix[i][j]=0
        
    index = pd.MultiIndex.from_arrays(all_pos_states, names=list(string.ascii_uppercase[:number_nodes]))
    columns = pd.MultiIndex.from_arrays(all_pos_states, names=list(string.ascii_uppercase[:number_nodes]))
    TPM = pd.DataFrame(matrix,columns=columns, index=index)
    
    return TPM 


## Cause Repertoire calculation:

The probability distribution of potential past states of a system as constrained by a mechanism in its current state.

In [337]:
def cause_repertoire(mechanism,purview, state, tpm):
    """
    Given the 'tpm' of a system, this function returns the cause repertoire of a 'mechanism' over a 'purview', 
    in a certian system's 'state'. Note: Comments on the next function (effect_repertoire()) may apply to this one too.
    Example: TPM=tpm(2,3,[0,1]), mechanism=['B'], purview=['A'], state={'A':1,'B':0,'C':0 },
    cause_repertoire(mechanism,purview, state, TPM).
    """ 
    
    factor=np.power(tpm.shape[0],1/len(state))**(len(tpm.index.names)-len(purview))  
    #how long the prob. will be given the number of purview's elemnts
    factor2=np.power(np.power(tpm.shape[0],1/len(state)), len(purview))
    
    #group in revrse [::-1] just to preserve paper's notation
    #i.e. the fisrt variable varies faster [A, B] A: 0101 B:0011(slower)
    tpm=(tpm.groupby(purview[::-1]).sum())*(1/factor)
    
    if len(mechanism)>0:
        tpm=(tpm.transpose().groupby(mechanism).sum()).transpose()
        col=[state[i] for i in mechanism]
        return list(tpm.loc[:, tuple(col)]/sum(list(tpm.loc[:, tuple(col)]))) if len(mechanism)>1\
                                                        else list(tpm.loc[:, col[0]]/sum(list(tpm.loc[:, col[0]])))
    else:
        #the unconstraind is just the normal distribution (equal probability for all)
        return [1/factor2]*int(factor2)

## Effect Repertoire calculation:

The probability distribution of potential future states of a system as constrained by a mechanism in its current state.

In [207]:
def effect_repertoire(mechanism,purview, state, tpm):
    """
    Given the 'tpm' of a system, this function returns the effect repertoire of a 'mechanism' over a 'purview', 
    in a certian system's 'state'. The purview needs to have one element only (a product is needed otherwise).
    Example: TPM=tpm(2,3,[0,1]), mechanism=['B'], purview=['A'], state={'A':1,'B':0,'C':0 },
    effect_repertoire(mechanism,purview, state, TPM).
    """ 

    #if n variables are marginalized from the mechanism, it implies that 'number of states' rows 
    #have to be added n times. Therefore, the result has to be divided by 1/('number of states'^n)= 1/factor.
    #np.power(tpm.shape[0],1/len(state))=number of states | (len(tpm.index.names)-len(mechanism))= n
    factor=np.power(tpm.shape[0],1/len(state))**(len(tpm.index.names)-len(mechanism))  
    
    #marginalize purview
    tpm=(tpm.transpose().groupby(purview).sum()).transpose()    

    # marginalize set of mechanisms
    if len(mechanism)>0:
        tpm=(tpm.groupby(mechanism).sum())*(1/factor)
        row=[state[i] for i in mechanism]
        return list(tpm.loc[tuple(row), :]) if len(mechanism)>1  else list(tpm.loc[row[0], :])
    else:
        return list(tpm.sum(axis=0)/tpm.shape[0])

## Calculation of the Tensor Product of two Probability Distributions.
[Math details](https://ncatlab.org/nlab/show/tensor+product+of+distributions)

In [176]:
def tensor_product(a,b):
    """
    It takes two lists a and b and outputs their tensor product as a list too.  
    """ 
    #transform the lists into dataframe for processing
    a=pd.DataFrame(a).transpose()
    b=pd.DataFrame(b).transpose()
    
    #calculate the tensor product
    matrix_prod=np.tensordot(a,b,axes=[0,0])
    #reshape the result into a one dimensional array (list)
    product=np.reshape(np.transpose(matrix_prod), matrix_prod.shape[0]*matrix_prod.shape[1]).tolist()
    
    return product

## Get the repertoires (cause or effect) of a mechanism over a purview.


In [300]:
def get_repertoire(mechanism, purview, state, tpm, rep_type, full_sys=True):
    """
    This function gets a repertoire 'rep_type'(cause or effect) of the 'mechanism' over the 'purview'
    in a system with 'state' given its 'tpm'. The variable 'full_sys' indicates whether or not,
    the resultant probability is expressed over the purview elements only, or over the entire state space.  
    """ 
    if not purview: return [1]
    L=list()
    state_space=list(string.ascii_uppercase[:len(state)])
    
    if rep_type=='effect':
        for element in state_space:
            if element in purview:
                L.append(effect_repertoire(mechanism,[element], state, tpm))
            elif full_sys: #calculate the unconstrained prob. over this non-purview element 
                L.append(effect_repertoire([],[element], state, tpm))
    elif rep_type=='cause':
        return cause_repertoire(mechanism,purview, state, tpm)
    else:
        print ('type of repertoire not recognized (cause|effect)')
    
    result=L[0]
    if len(L)>1: #the purvie had more than one element and tensor product has to be performed
        for i in range(len(L)-1):
            result=tensor_product(result,L[i+1])  
        
    return result
    

## EXAMPLES.


### Example 1:
Let's reproduce the results for a binary system as shown in the original paper [page 7](https://journals.plos.org/ploscompbiol/article/file?id=10.1371/journal.pcbi.1003588&type=printable).

![title](paper.png)


In [359]:
#define the TPM for a system '2' (see description at the very top) with three binay elemnts
TPM=tpm(2,3,[0,1])

Cause repertoire:

In [369]:
mechanism=['A'] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
list(np.around(np.array(ans),3))

[0.0, 0.0, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167]

Unconstrained past:

In [370]:
mechanism=[] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
get_repertoire(mechanism,purview, state,TPM, 'cause',True)

[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]

Effect repertoire:

In [371]:
mechanism=['A'] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
get_repertoire(mechanism,purview, state,TPM, 'effect',True)

[0.0625, 0.1875, 0.0625, 0.1875, 0.0625, 0.1875, 0.0625, 0.1875]

Unconstrained future:

In [372]:
mechanism=[] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
get_repertoire(mechanism,purview,state,TPM, 'effect',True)

[0.09375, 0.28125, 0.03125, 0.09375, 0.09375, 0.28125, 0.03125, 0.09375]

### Example 2:
Let's calculate some cause and effect repertoires for the non-binary system of three elements, 
whose description and graph are shown on the very top of this Jupyter Notebook:


In [373]:
TPM=tpm(1,3,[-1,0,1])
TPM

,,A,-1,0,1,-1,0,1,-1,0,1,-1,...,1,-1,0,1,-1,0,1,-1,0,1
,,B,-1,-1,-1,0,0,0,1,1,1,-1,...,1,-1,-1,-1,0,0,0,1,1,1
,,C,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,...,0,1,1,1,1,1,1,1,1,1
A,B,C,,,,,,,,,,,,,,,,,,,,,
-1,-1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,-1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1,0,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Cause repertoire of A/ABC

In [383]:
mechanism=['A'] 
purview=['A','B','C']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
ans=list(np.around(np.array(ans),1))
print(*ans) 

0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0


Cause repertoire of ABC/ABC

In [377]:
mechanism=['A','B','C'] 
purview=['A','B','C']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
ans=list(np.around(np.array(ans),1))
print(*ans) 

0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0


Cause repertoire of []/ABC

In [379]:
mechanism=[] 
purview=['A','B','C']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
ans=list(np.around(np.array(ans),3))
print(*ans) 

0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037


Effect repertoire of A/AB

In [381]:
mechanism=['A'] 
purview=['A','B']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'effect',True)
ans=list(np.around(np.array(ans),3))
print(*ans) 

0.074 0.074 0.074 0.037 0.037 0.037 0.0 0.0 0.0 0.074 0.074 0.074 0.037 0.037 0.037 0.0 0.0 0.0 0.074 0.074 0.074 0.037 0.037 0.037 0.0 0.0 0.0


In [385]:
mechanism=['A'] 
purview=['A','B']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'effect',False)
ans=list(np.around(np.array(ans),3))
print(*ans) 

0.222 0.222 0.222 0.111 0.111 0.111 0.0 0.0 0.0
